# 第9章 相关与回归及Python分析

## 9.1 变量间的关系

## 9.2 一元相关与回归分析

### 9.2.1 简单相关分析

In [ ]:
#%run init.py

In [ ]:
import numpy as np
np.set_printoptions(4) 
x=np.array([171,175,159,155,152,158,154,164,168,166,159,164])  #身高向量
y=np.array([57,64,41,38,35,44,41,51,57,49,47,46])              #体重向量

In [ ]:
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt              #加载基本绘图包
plt.scatter(x,y);                            #做散点图
plt.xlabel('x');plt.ylabel('y');

In [ ]:
def lxy(x,y):return sum(x*y)-sum(x)*sum(y)/len(x)  #离均差乘积和函数

In [ ]:
lxy(x,x)  #x的离均差平方和

In [ ]:
lxy(y,y)  #y的离均差平方和

In [ ]:
lxy(x,y)  #x和y的离均差乘积和

In [ ]:
r=lxy(x,y)/np.sqrt(lxy(x,x)*lxy(y,y)) ;r  #相关系数计算

In [ ]:
#np.corrcoef(x,y)       #计算相关系数

In [ ]:
n=len(x)                  #向量的长度
tr=r/np.sqrt((1-r**2)/(n-2))  #相关系数假设检验t统计量
tr

In [ ]:
#from scipy import stats as st             #加载统计包
import scipy.stats as st             #加载统计包
rp=st.pearsonr(x,y)
print('r=%.4f  p-value=%.4g'%(rp[0],rp[1]))

### 9.2.2 简单回归分析

In [ ]:
b=lxy(x,y)/lxy(x,x)             #线性回归方程斜率
a=y.mean()-b*x.mean()           #线性回归方程截距
print('a=%.4f  b=%.4f'%(a,b))   #显示线性回归方程估计值

In [ ]:
plt.plot(x,y,'o',x,a+b*x,'-');     #添加估计方程线
plt.xlabel('x');plt.ylabel('y');

In [ ]:
import pandas as pd                          #加载数据分析包
pd.set_option('display.precision',4)         #设置数据框输出精度
d93=pd.read_excel('mvsData.xlsx','d93',index_col=0);d93 

In [ ]:
from statsmodels.formula.api import ols   #加载公式法普通最小二乘函数
fm0=ols('y ~ x',data=d93).fit()           #拟合一元线性回归模型
fm0.params                                #显示回归参数

In [ ]:
plt.plot(d93.x,d93.y,'o',d93.x,fm0.fittedvalues,'-');  #添加回归线
plt.xlabel('x');plt.ylabel('y');

In [ ]:
#import statsmodels.api as sm   #加载统计分析模型包
#sm.stats.anova_lm(fm0)          #模型方差分析

In [ ]:
print(fm0.summary().tables[1])  #回归系数t检验

## 9.3 多元相关与回归分析

### 9.3.1 多元线性相关分析

In [ ]:
d94=pd.read_excel('mvsData.xlsx','d94',index_col=0); d94 

In [ ]:
print(d94.corr())  #多元数据相关系数矩阵

In [ ]:
%config InlineBackend.figure_format='retina'   #可提高图形显示的清晰度
pd.plotting.scatter_matrix(d94);  #多元数据散点图

In [ ]:
import scipy.stats as st    #加载统计包
def mcor_test(X):     #相关系数矩阵检验
    p=X.shape[1];p
    sp=np.ones([p, p]).astype(str)
    for i in range(0,p):
        for j in range(i,p):        
            P=st.pearsonr(X.iloc[:,i],X.iloc[:,j])[1]        
            if P>0.05: sp[i,j]=' '
            if(P>0.01 and P<=0.05): sp[i,j]='*'
            if(P>0.001 and P<=0.01): sp[i,j]='**'
            if(P<=0.001): sp[i,j]='***'
            r=st.pearsonr(X.iloc[:,i],X.iloc[:,j])[0]
            sp[j,i]=round(r,4)
            if(i==j):sp[i,j]='------'    
    print(pd.DataFrame(sp,index=X.columns,columns=X.columns))
    print("\n下三角为相关系数，上三角为检验p值 * p<0.05 ** p<0.05 *** p<0.001")

In [ ]:
mcor_test(d94)   #多元数据相关系数检验

### 9.3.2 多元线性回归模型的建立

In [ ]:
fm=ols('y~x1+x2+x3+x4',data=d94).fit()
fm.params              #模型参数

In [ ]:
#from scipy import stats             
#fm1_z=ols('y ~ x1+x2+x3+x4',data=d9_4.apply(stats.zscore)).fit()
#fm1_z.params  #标准回归系数

### 9.3.3 多元线性回归模型的检验

In [ ]:
print(fm.summary())  #多元线性回归系数t检验

In [ ]:
fm1=ols('y~x1',data=d94).fit();fm1.params

In [ ]:
fm2=ols('y~x1+x2',data=d94).fit();fm2.params

In [ ]:
fm3=ols('y~x1+x2+x3',data=d94).fit();fm3.params

In [ ]:
fm4=ols('y~x1+x2+x3+x4',data=d94).fit();fm4.params

In [ ]:
from statsmodels.iolib.summary2 import summary_col
summary_col([fm1,fm2,fm3,fm4])

### 9.3.4 复相关及模型的决定系数

In [ ]:
R2=fm.rsquared
print('模型的决定系数 R^2 = %5.4f'%R2)

In [ ]:
R=np.sqrt(R2);R    
print('变量的复相关系数 R = %5.4f'%np.sqrt(R2))

## 案例9：中国宏观经济发展的多因素分析

In [ ]:
Case9=pd.read_excel('mvsCase.xlsx','Case9'); Case9

In [ ]:
#Case9.describe()

In [ ]:
mcor_test(Case9)   #相关系数两两检验

In [ ]:
pd.plotting.scatter_matrix(Case9);  #矩阵散点图

In [ ]:
from statsmodels.formula.api import ols   
print(ols('Y~X1+X2+X3+X4+X5+X6',data=Case9).fit().summary())

In [ ]:
fm=ols('Y~X1+X2+X4',data=Case9).fit();
print(fm.summary())

In [ ]:
plt.figure(figsize=(7,5)) 
plt.plot(Case9.Y,'o',Case9.index,fm.fittedvalues,'-');